# Simulation using the `pulser.sampler` module

This notebook is a work document to investigate how to use the sampler correctly in the simulation module.

In [1]:
import numpy as np

import pulser
import pulser_simulation
from pulser import Pulse
from pulser.devices import MockDevice
from pulser.waveforms import BlackmanWaveform

In [2]:
reg = pulser.Register.square(1)
seq = pulser.Sequence(reg, MockDevice)

seq.declare_channel("main", "rydberg_global")

seq.add(Pulse.ConstantPulse(500, 10, 20, 0), "main")

In [3]:
sim = pulser_simulation.Simulation(seq)
res = sim.run()
print(len(sim.samples["Global"]["ground-rydberg"]["amp"]))
fs1 = res.get_final_state()
print(fs1)

Using legacy sampling
501
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.17230094+0.22978882j]
 [0.95786715+0.j        ]]


In [4]:
config2 = pulser_simulation.SimConfig(use_sampler=True)
sim2 = pulser_simulation.Simulation(seq, config=config2)
res2 = sim2.run()
print(len(sim2.samples["Global"]["ground-rydberg"]["amp"]))
fs2 = res2.get_final_state()
print(fs2)

Using the sampler
500
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.17041078+0.22931142j]
 [0.95831959+0.j        ]]


No more basis problem. The final state differ, most probably because of the additional zero sample in the legacy sampler.

In [5]:
all(
    sim.samples["Global"]["ground-rydberg"]["amp"][:500]
    - sim2.samples["Global"]["ground-rydberg"]["amp"]
    == 0
)

True

In [6]:
print(sim.samples["Global"]["ground-rydberg"]["amp"][-10:])
print(sim2.samples["Global"]["ground-rydberg"]["amp"][-10:])

[10. 10. 10. 10. 10. 10. 10. 10. 10.  0.]
[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]


In [7]:
print(fs1 - fs2)

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.00189016+0.00047741j]
 [-0.00045244+0.j        ]]


This should not be a problem for pulse waveforms that goes to zeros.
Let's check.

In [8]:
seq_blackman = pulser.Sequence(reg, MockDevice)
seq_blackman.declare_channel("main", "rydberg_global")
seq_blackman.add(
    Pulse.ConstantDetuning(BlackmanWaveform(500, np.pi / 2), 0, 0), "main"
)

sim3 = pulser_simulation.Simulation(seq_blackman)
res3 = sim3.run()

sim4 = pulser_simulation.Simulation(
    seq_blackman, config=pulser_simulation.SimConfig(use_sampler=True)
)
res4 = sim4.run()

Using legacy sampling
Using the sampler


In [9]:
print(res3.get_final_state())
print(res4.get_final_state())

np.testing.assert_allclose(
    res3.get_final_state(), res4.get_final_state(), rtol=0, atol=1e-7
)

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.      -0.70708856j]
 [0.707125+0.j        ]]
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.        -0.70708855j]
 [0.70712502+0.j        ]]


The `digital` key still exist in the sample dictionary with the new sampler, it is just zero.

In [10]:
print(sim2.samples["Global"].keys())
print(sim2.samples["Global"]["digital"])

dict_keys(['ground-rydberg', 'digital'])
{'amp': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 